In [37]:
import os
import pathlib

import earthaccess
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import pandas as pd
import shapely

# Satellite tracks for test site on Willamette River
passes = [39, 274, 345]

## Find node ID's from earthaccess

In [2]:
earthaccess.login()

In [3]:
# Search for the SWOT dataset
swot_datasets = earthaccess.search_datasets(keyword="SWOT Level 2 Vector")
for dataset in swot_datasets:
    try:
        print(dataset['umm']['CollectionCitations'][0]['Title'], 
              dataset['umm']['ShortName'])
    except:
        print('Dataset missing title information')

SWOT Level 2 River Single-Pass Vector Data Product, Version C SWOT_L2_HR_RiverSP_2.0
SWOT Level 2 Lake Single-Pass Vector Data Product, Version C SWOT_L2_HR_LakeSP_2.0
SWOT Level 2 Lake Single-Pass Vector Data Product, Version D SWOT_L2_HR_LakeSP_D
SWOT Level 2 Lake Single-Pass Vector Obs Data Product, Version C SWOT_L2_HR_LakeSP_obs_2.0
SWOT Level 2 Lake Single-Pass Vector Obs Data Product, Version D SWOT_L2_HR_LakeSP_obs_D
SWOT Level 2 Lake Single-Pass Vector Prior Data Product, Version C SWOT_L2_HR_LakeSP_prior_2.0
SWOT Level 2 Lake Single-Pass Vector Prior Data Product, Version D SWOT_L2_HR_LakeSP_prior_D
SWOT Level 2 Lake Single-Pass Vector Unassigned Data Product, Version C SWOT_L2_HR_LakeSP_unassigned_2.0
SWOT Level 2 Lake Single-Pass Vector Unassigned Data Product, Version D SWOT_L2_HR_LakeSP_unassigned_D
SWOT Level 2 River Single-Pass Vector Data Product, Version D SWOT_L2_HR_RiverSP_D
SWOT Level 2 River Single-Pass Vector Node Data Product, Version C SWOT_L2_HR_RiverSP_node_2

In [25]:
vector_meta = earthaccess.search_data(
    short_name="SWOT_L2_HR_RiverSP_node_D",
    bounding_box=(-124, 40, -122, 45),
)
willamette_meta = [
    meta for meta in vector_meta 
    if meta['umm']['SpatialExtent']['HorizontalSpatialDomain']['Track']['Passes'][0]['Pass'] in passes]
willamette_meta

[Collection: {'Version': 'D', 'ShortName': 'SWOT_L2_HR_RiverSP_node_D'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -132.82844664341883, 'SouthBoundingCoordinate': 34.27537895840783, 'EastBoundingCoordinate': -119.11105290260366, 'NorthBoundingCoordinate': 60.14599055702098}]}, 'Track': {'Cycle': 32, 'Passes': [{'Pass': 274, 'Tiles': ['48L', '49L', '50L', '51L', '52L', '53L', '54L', '55L', '56L', '57L', '58L', '59L', '60L', '61L', '62L', '63L', '64L', '65L', '66L', '67L', '68L', '69L', '70L', '71L', '72L', '73L', '74L', '75L', '76L', '77L', '78L', '79L', '80L', '81L', '82L', '83L', '84L', '85L', '86L', '87L', '88L', '89L', '90L', '91L', '92L', '93L', '94L', '95L', '96L', '97L', '98L', '99L', '100L', '101L', '102L', '103L', '104L', '48R', '49R', '50R', '51R', '52R', '53R', '54R', '55R', '56R', '57R', '58R', '59R', '60R', '61R', '62R', '63R', '64R', '65R', '66R', '67R', '68R', '69R', '70R', '71R', '72R', '73R', '74R', '75

In [26]:

files = earthaccess.open(willamette_meta)
files

QUEUEING TASKS | : 100%|██████████| 6/6 [00:00<00:00, 579.04it/s]
PROCESSING TASKS | : 100%|██████████| 6/6 [00:02<00:00,  2.43it/s]
COLLECTING RESULTS | : 100%|██████████| 6/6 [00:00<00:00, 36578.23it/s]


[<File-like object HTTPFileSystem, https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_HR_RiverSP_D/SWOT_L2_HR_RiverSP_Node_032_274_NA_20250507T190407_20250507T191154_PID0_03.zip>,
 <File-like object HTTPFileSystem, https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_HR_RiverSP_D/SWOT_L2_HR_RiverSP_Node_032_345_NA_20250510T082716_20250510T083046_PID0_01.zip>,
 <File-like object HTTPFileSystem, https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_HR_RiverSP_D/SWOT_L2_HR_RiverSP_Node_033_039_NA_20250520T064829_20250520T065254_PID0_01.zip>,
 <File-like object HTTPFileSystem, https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_HR_RiverSP_D/SWOT_L2_HR_RiverSP_Node_033_274_NA_20250528T154914_20250528T155701_PID0_01.zip>,
 <File-like object HTTPFileSystem, https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_HR_R

In [58]:
swot_gdf_list = []
for meta, file in zip(willamette_meta, files):
    gdf = gpd.read_file(file)
    gdf['pass'] = str(
        meta['umm']['SpatialExtent']['HorizontalSpatialDomain']['Track']['Passes'][0]['Pass'])
    swot_gdf_list.append(gdf)
swot_gdf = pd.concat(swot_gdf_list)

In [61]:
bbox = shapely.geometry.box(
    minx=-123.4, 
    miny=44.15, 
    maxx=-123, 
    maxy=44.5)
site_gdf = swot_gdf.clip(bbox)
site_gdf.hvplot(
    geo=True, tiles='CartoLight',
    frame_width=250, frame_height=500)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

## Download MRMS data

## Download SWOT time series

In [ ]:
for